# Extract the frames out of the videos and set up the classifcation framework

## Setup

In [ ]:
import os
import cv2
import shutil
import random
import json
import time

### Setup Global Variables

In [ ]:
# Declareer globale variabelen
global interval 

# Initialiseer de variabelen
interval = 3

### The table data met labels (copy of the collect data)

In [ ]:
# Create an array containing the table data with an index
table_data = [
    ["Label", "Gaze Direction", "Sentence"],
    ["forward", "Forward", "Look forward."],
    ["left", "Left", "Look to the left."],
    ["right", "Right", "Look to the right."],
    ["mirror_interior", "Interior Mirror", "Look at the interior mirror."],
    ["mirror_right", "Right Side Mirror", "Look at the right side mirror."],
    ["mirror_left", "Left Side Mirror", "Look at the left side mirror."],
    ["shoulder_right", "Right Shoulder", "Look over your right shoulder."],
    ["shoulder_left", "Left Shoulder", "Look over your left shoulder."],
    ["dashboard_straight_down", "Dashboard Straight Down", "Look straight down at the dashboard."],
    ["dashboard_down_right", "Dashboard Down Towards Center Console", "Look down towards the center console."],
    ["forward_right", "Forward Right", "Look forward and to the right."],
    ["forward_left", "Forward Left", "Look forward and to the left."]
]

# To access, for example, the row for "forward_right" and its Sentence column:
# Note that Python is 0-indexed, so row 11 (forward_right) is at index 10 in the array
# print(table_data[11][2])  # Output: Look forward and to the right.


## Iterate over the collected data, call the extract function and divide the files on the directories train, test and val. 

### Interate over the labels and ranges

In [ ]:
base_path = r"C:\GazeDetection\test_recording_correct_upload_4"
metadata_list = []
frame_counts = {}  # Dictionary om de frame tellingen per dataset type bij te houden

for row in table_data[1:]:  # Header overslaan
    label = row[0]
    print(f"Label: {label}")
    output_dir = os.path.join(base_path, "train", label)
    frame_counts[label] = {'train': 0, 'test': 0, 'val': 0}  # Initialisatie van tellingen

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in range(1, 11):  # Aanname dat er 10 video's per label zijn
        file_dir = os.path.join(base_path, label, str(i))
        for camera in [1, 2]:
            video_path = os.path.join(file_dir, f"camera_{camera}.avi")
            metadata = extract_frames(video_path, output_dir, camera, 5, i, label)  # Aannemend dat extract_frames correct is gedefinieerd
            if metadata:
                metadata_list.append(metadata)
                frame_counts[label]['train'] += metadata['extracted_frame_count']  # Aantal geëxtraheerde frames optellen bij train

    # Bepaal hoeveel bestanden verplaatst moeten worden (10%)
    files = os.listdir(output_dir)
    num_files_to_move = max(1, len(files) // 10)

    # Maak directories als ze nog niet bestaan
    test_directory = os.path.join(base_path, "test", label)
    val_directory = os.path.join(base_path, "val", label)
    os.makedirs(test_directory, exist_ok=True)
    os.makedirs(val_directory, exist_ok=True)

    # Verplaats bestanden naar 'test' en 'val'
    frame_counts[label]['test'] = divide(output_dir, test_directory, num_files_to_move)
    frame_counts[label]['val'] = divide(output_dir, val_directory, num_files_to_move)
    print(f"Label: {label}, Train: {len(files) - num_files_to_move*2}, Test: {frame_counts[label]['test']}, Val: {frame_counts[label]['val']}")

# Schrijf metadata en frame tellingen naar de readme.txt file
readme_path = os.path.join(base_path, "readme.txt")
with open(readme_path, 'w') as f:
    for meta in metadata_list:
        f.write(json.dumps(meta, indent=4) + '\n')
    f.write("\nFrame Counts by Label:\n")
    for label, counts in frame_counts.items():
        # Gebruik enkele aanhalingstekens binnen de os.path.join om conflicten te vermijden
        train_files_dir = os.path.join(base_path, 'train', label)
        train_count = len(os.listdir(train_files_dir))
        f.write(f"Label: {label}, Train: {train_count}, Test: {counts['test']}, Val: {counts['val']}\n")

### Extract de frames out the videos and save in the output_dir

In [ ]:
def extract_frames(video_path, output_dir, camera, interval, frequentie, label):
    # Initialize the video capture object
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None

    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Initialize frame index and list to store extracted frames
    frame_idx = 0
    extracted_frames_count = 0

    # Process the video and extract frames
    while frame_idx < total_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % interval == 0:
            filename = f"{label}_camera_{camera}_cycle_{frequentie}_frame_{frame_idx}.jpg"
            frame_file = os.path.join(output_dir, filename)
            cv2.imwrite(frame_file, frame)
            extracted_frames_count += 1
        frame_idx += 1

    # Release the video capture object
    cap.release()
    
    # Determine the time and date from file modification time
    mod_time = os.path.getmtime(video_path)
    time_date_recorded = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(mod_time))

    # Return metadata for the video
    metadata = {
        'filename': os.path.join(label, os.path.basename(video_path)),
        'time_date_recorded': time_date_recorded,
        'fps': fps,
        'width': width,
        'height': height,
        'frame_count': total_frames,
        'extracted_frame_count': extracted_frames_count
    }
    return metadata


### Moves an percentage of the files from one folder to another.

In [ ]:
def divide(source_dir, target_dir, num_files_to_move):
    files = random.sample(os.listdir(source_dir), num_files_to_move)
    for file in files:
        shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, file))
    return len(files)